# Overview

Last week, Dr Rachel Sippy and I decided on the following:

- Project is to be split in two parts
- Part 1: Extracting locations from tweet text, and from ProMed. See how many line up, and that is the Masters Project.
- Part 2: Using time series model to determine where dengue is coming from.

Data format:

**From Twitter**

Epi. Week, Location, username, Year

Epi. Weelk, Location, No. users, Year

**Dengue**

Epi. Week, Locations, Year

In [2]:
import pandas as pd
import boto3

df = pd.read_json("s3://mt5599/tweets/handles_combined.json")
df

,id,DateTime,tweet_content,cashtags,coordinates,hashtags,lang,like_count,media,mentioned_users,...,user_profile_image_url,user_profile_banner_url,user_label,place_country,place_country_code,place_full_name,place_name,place_type,coordinates_longitude,coordinates_latitude
0,671493915590057984,2015-12-01 00:59:59,mi fin de semana estuvo cabron,None,"Coordinates(longitude=-67.054105, latitude=18....",None,es,1,None,None,...,https://pbs.twimg.com/profile_images/160177785...,https://pbs.twimg.com/profile_banners/16040915...,None,United States,US,"Bejucos, Puerto Rico",Bejucos,city,-67.054105,18.481387
1,671493915497885696,2015-12-01 00:59:59,Ya me descarge el app😁 ahora mati no me va a p...,None,"Coordinates(longitude=-58.346384, latitude=-34...",None,es,1,None,None,...,https://pbs.twimg.com/profile_images/153241012...,https://pbs.twimg.com/profile_banners/22690117...,None,Argentina,AR,"Quilmes, Argentina",Quilmes,city,-58.346384,-34.801165
2,671493915342688256,2015-12-01 00:59:59,@gabyespinony Dejame buscar y te aviso !!!,None,"Coordinates(longitude=-73.3540725, latitude=0....",None,es,0,None,"[User(username='gabyespinony', id=2769738383, ...",...,https://pbs.twimg.com/profile_images/152098656...,None,None,Venezuela,VE,Venezuela,Venezuela,country,-73.354073,0.648837
3,671493914801643520,2015-12-01 00:59:59,Sin dudas la mejor gestión,None,"Coordinates(longitude=-64.3195, latitude=-31.5...",None,es,0,None,None,...,https://pbs.twimg.com/profile_images/160430215...,https://pbs.twimg.com/profile_banners/43886449...,None,Argentina,AR,"Córdoba, Argentina",Córdoba,city,-64.319500,-31.523576
4,671493914176724992,2015-12-01 00:59:59,Elbo Lita jajajaja que cldo!,None,"Coordinates(longitude=-64.389839, latitude=-33...",None,es,0,None,None,...,https://pbs.twimg.com/profile_images/140897405...,https://pbs.twimg.com/profile_banners/22708841...,None,Argentina,AR,"Río Cuarto, Argentina",Río Cuarto,city,-64.389839,-33.159419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655961,792863655096442880,2016-10-30 23:00:00,"@CFKArgentina con las propiedades de Báez,cua...",None,"Coordinates(longitude=-63.2919697, latitude=-3...",None,es,0,None,"[User(username='CFKArgentina', id=138814032, d...",...,https://pbs.twimg.com/profile_images/149981312...,https://pbs.twimg.com/profile_banners/76809874...,None,Argentina,AR,"General San Martín, Argentina",General San Martín,city,-63.291970,-32.432099
1655962,792863654274367488,2016-10-30 23:00:00,"Temperatura: 22,4°C S.Térmica 22,4°C PR 12,7°C...",None,"Coordinates(longitude=-58.54916667, latitude=-...",None,es,1,None,None,...,https://pbs.twimg.com/profile_images/160486963...,https://pbs.twimg.com/profile_banners/47917531...,None,Argentina,AR,"Ciudad del Libertador General San Martín, Arge...",Ciudad del Libertador General San Martín,city,-58.549167,-34.535278
1655963,792863654089723904,2016-10-30 23:00:00,"que ondon Ramón con el camarón, que hongo joro...",None,"Coordinates(longitude=-97.43391558, latitude=3...",None,es,0,None,None,...,https://pbs.twimg.com/profile_images/839951285...,https://pbs.twimg.com/profile_banners/80163931...,None,United States,US,"Texas, USA",Texas,admin,-97.433916,30.346318
1655964,792863653401722880,2016-10-30 23:00:00,Extrañaba esto ☀ @ Nacimiento. Región Bio Bio ...,None,"Coordinates(longitude=-72.6704209, latitude=-3...",None,es,0,None,None,...,https://pbs.twimg.com/profile_images/760588784...,https://pbs.twimg.com/profile_banners/21230792...,None,Chile,CL,"Nacimiento, Chile",Nacimiento,city,-72.670421,-37.501660


In [7]:
df.place.unique()

AttributeError: 'DataFrame' object has no attribute 'place'

In [9]:
import pandas as pd

df = pd.read_feather("../data/df_2016_tweets_cleaned.feather")
df

,id,DateTime,tweet_content,cashtags,coordinates,hashtags,lang,like_count,media,mentioned_users,...,user_listed_count,user_media_count,user_protected,user_link_url,user_link_tcourl,user_profile_image_url,user_profile_banner_url,user_label,resident/visitor,tweet
0,743561801745338368,2016-06-16 21:52:02,😁 https://t.co/eq2i7lxQkD,None,"Coordinates(longitude=-85.3044586, latitude=8....",None,und,1,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,2,215,False,None,None,https://pbs.twimg.com/profile_images/154994465...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,
1,743064119754919936,2016-06-15 12:54:26,Morning ☕️🌲🍂🌾🍃,None,"Coordinates(longitude=-85.3044586, latitude=8....",None,en,0,None,None,...,2,215,False,None,None,https://pbs.twimg.com/profile_images/154994465...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,morning
2,742934385624809472,2016-06-15 04:18:55,👬👭❤️🤗 https://t.co/nMQNrePMXZ,None,None,None,und,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,2,215,False,None,None,https://pbs.twimg.com/profile_images/154994465...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,
3,726430125286264832,2016-04-30 15:16:52,Se van los 25 :/ https://t.co/RddGa6gT4T,None,None,None,es,3,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,2,215,False,None,None,https://pbs.twimg.com/profile_images/154994465...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,se van los 25 :/
4,684363985974464512,2016-01-05 13:21:03,Godo Morning 👓,None,"Coordinates(longitude=-84.851172, latitude=9.9...",None,en,1,None,None,...,2,215,False,None,None,https://pbs.twimg.com/profile_images/154994465...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,godo morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7686538,649582930189156352,2015-10-01 13:53:33,El bus va vacío pero no me voy a sentar com t...,None,"Coordinates(longitude=-84.2022247, latitude=9....",None,es,0,None,None,...,12,2087,False,None,None,https://pbs.twimg.com/profile_images/157661616...,https://pbs.twimg.com/profile_banners/84109116...,None,resident,el bus va vaco pero no me voy a sentar com tal...
7686539,649561708587954176,2015-10-01 12:29:14,Buenos días ! \nbienvenido octubre 👏🏼,None,"Coordinates(longitude=-85.4422302, latitude=9....",None,es,0,None,None,...,12,2087,False,None,None,https://pbs.twimg.com/profile_images/157661616...,https://pbs.twimg.com/profile_banners/84109116...,None,resident,buenos das ! bienvenido octubre
7686540,649446900601483264,2015-10-01 04:53:01,Y terminar comiendo una Queso de Mc 😊 😋🍔,None,None,None,es,0,None,None,...,12,2087,False,None,None,https://pbs.twimg.com/profile_images/157661616...,https://pbs.twimg.com/profile_banners/84109116...,None,resident,y terminar comiendo una queso de mc
7686541,649435258853527552,2015-10-01 04:06:46,Gracias Mae por su culpa estoy que me vomito 😤,None,"Coordinates(longitude=-84.1299362, latitude=9....",None,es,0,None,None,...,12,2087,False,None,None,https://pbs.twimg.com/profile_images/157661616...,https://pbs.twimg.com/profile_banners/84109116...,None,resident,gracias mae por su culpa estoy que me vomito


## Available Geoparsers

GeoTxt
(API not available)

Edinburgh Geoparser

https://www.ltg.ed.ac.uk/software/geoparser/

CLAVIN

https://github.com/Novetta/CLAVIN

CamCoder

https://github.com/milangritta/Geocoding-with-Map-Vector

https://www.repository.cam.ac.uk/handle/1810/277772